In [ ]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1', 'https://qlawsdl001038a.ad.datalake.foc.zone:8443/gateway/dsx/livy/v1']


In [ ]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g",
 "driverMemory": "8g", "proxyUser": "jlu-", "driverCores": 1,
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [ ]:
%spark add -s jlu -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
36257,application_1617092791322_291414,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, countDistinct, max, min, desc
from pyspark.sql import DataFrameStatFunctions as statFunc
import pyspark.sql.types as T
from functools import reduce


from os.path import expanduser, join, abspath
import time
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

spark = SparkSession.builder.getOrCreate()

In [ ]:
%%spark

from pyspark.sql.functions import lower

csv_path = '/dev/projects/retention_models/csv_data/'
actual_path = '/dev/projects/retention_models/actual_value/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'

past_path = '/dev/projects/retention_models/purchase_payoff/training/results/movermodel/'

In [ ]:
%%spark
# used to be 20210201
df_actual = spark.read.csv(csv_path + 'Payoff_segments_summary_20210405.csv', header=True)\
                        .withColumn('newloanpurpose', lower(col('NewLoanPurpose')))\
                        .withColumn('retentiontype', lower(col('PayoffRetentionType')))\
                        .select('ServicedLoanNumber', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype')

In [ ]:
%%spark
df_actual.orderBy('PaymentInFullDate').show(5)

+------------------+-----------------+--------------+-------------+
|ServicedLoanNumber|PaymentInFullDate|newloanpurpose|retentiontype|
+------------------+-----------------+--------------+-------------+
|        3222053188|       2010-10-01|       unknown|         lost|
|        3221001169|       2010-10-28|     refinance|     retained|
|        3223266428|       2010-11-18|     refinance|     retained|
|        3224646047|       2010-12-30|       unknown|         lost|
|        3224047755|       2010-12-31|       unknown|         lost|
+------------------+-----------------+--------------+-------------+
only showing top 5 rows

In [ ]:
%%spark

# df_list1 = spark.sql('''
# select distinct servicecalendardate, ln_no
# from data_science_sandbox.servicing_dbo_payoff_source_purchase_client_crdtrs
# where leftportfoliodate is null
# and servicecalendardate between '2016-06-30' and '2018-11-30'
# ''')

## upload from DW table
## add more snapshot to update the current servicing loans payoff information

df_201801 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201801.parquet').select('servicecalendardate', 'ln_no')
df_201802 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201802.parquet').select('servicecalendardate', 'ln_no')
df_201803 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201803.parquet').select('servicecalendardate', 'ln_no')
df_201804 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201804.parquet').select('servicecalendardate', 'ln_no')
df_201805 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201805.parquet').select('servicecalendardate', 'ln_no')
df_201806 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201806.parquet').select('servicecalendardate', 'ln_no')
df_201807 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201807.parquet').select('servicecalendardate', 'ln_no')
df_201808 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201808.parquet').select('servicecalendardate', 'ln_no')
df_201809 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201809.parquet').select('servicecalendardate', 'ln_no')
df_201810 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201810.parquet').select('servicecalendardate', 'ln_no')
df_201811 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201811.parquet').select('servicecalendardate', 'ln_no')
df_201812 = spark.read.parquet(monthly_path+'monthly_preprocessed/monthly_preprocessed_201812.parquet').select('servicecalendardate', 'ln_no')

df_201901 = spark.read.parquet(monthly_path+'servicing_df_all_jan19_20190215.parquet').select('servicecalendardate', 'ln_no')
df_201902 = spark.read.parquet(monthly_path+'servicing_df_all_feb19_20190305.parquet').select('servicecalendardate', 'ln_no')
df_201903 = spark.read.parquet(monthly_path+'servicing_df_all_mar19_20190409.parquet').select('servicecalendardate', 'ln_no')
df_201904 = spark.read.parquet(monthly_path+'servicing_df_all_apr19_20190507.parquet').select('servicecalendardate', 'ln_no')
df_201905 = spark.read.parquet(monthly_path+'servicing_df_all_may19_20190610.parquet').select('servicecalendardate', 'ln_no')
df_201906 = spark.read.parquet(monthly_path+'servicing_df_all_june19_20190708.parquet').select('servicecalendardate', 'ln_no')
df_201907 = spark.read.parquet(monthly_path+'servicing_df_all_jul19_20190805.parquet').select('servicecalendardate', 'ln_no')
df_201908 = spark.read.parquet(monthly_path+'servicing_df_all_aug19_20191226.parquet').select('servicecalendardate', 'ln_no')
df_201909 = spark.read.parquet(monthly_path+'servicing_df_all_sep19_20191004.parquet').select('servicecalendardate', 'ln_no')
df_201910 = spark.read.parquet(monthly_path+'servicing_df_all_oct19_20191105.parquet').select('servicecalendardate', 'ln_no')
df_201911 = spark.read.parquet(monthly_path+'servicing_df_all_nov19_20191202.parquet').select('servicecalendardate', 'ln_no')
df_201912 = spark.read.parquet(monthly_path+'servicing_df_all_dec19_20200103.parquet').select('servicecalendardate', 'ln_no')

df_202001 = spark.read.parquet(monthly_path+'servicing_df_all_jan20_20200203.parquet').select('servicecalendardate', 'ln_no')
df_202002 = spark.read.parquet(monthly_path+'servicing_df_all_feb20_20200305.parquet').select('servicecalendardate', 'ln_no')
df_202003 = spark.read.parquet(monthly_path+'servicing_df_all_mar20_20200406.parquet').select('servicecalendardate', 'ln_no')
df_202004 = spark.read.parquet(monthly_path+'servicing_df_all_apr20_20200505.parquet').select('servicecalendardate', 'ln_no')
df_202005 = spark.read.parquet(monthly_path+'servicing_df_all_may20_20200610.parquet').select('servicecalendardate', 'ln_no')
df_202006 = spark.read.parquet(monthly_path+'servicing_df_all_june20_20200709.parquet').select('servicecalendardate', 'ln_no')
df_202007 = spark.read.csv(csv_path+'servicing_loanlist_202007.csv', header = True).select('servicecalendardate', 'ln_no')
df_202008 = spark.read.csv(csv_path+'servicing_loanlist_202008.csv', header = True).select('servicecalendardate', 'ln_no')
df_202009 = spark.read.csv(csv_path+'servicing_loanlist_202009.csv', header = True).select('servicecalendardate', 'ln_no')
df_202010 = spark.read.csv(csv_path+'servicing_loanlist_202010.csv', header = True).select('servicecalendardate', 'ln_no')
df_202011 = spark.read.csv(csv_path+'servicing_loanlist_202011.csv', header = True).select('servicecalendardate', 'ln_no')
df_202012 = spark.read.csv(csv_path+'servicing_loanlist_202012.csv', header = True).select('servicecalendardate', 'ln_no')

df_202101 = spark.read.csv(csv_path+'servicing_loanlist_202101.csv', header = True).select('servicecalendardate', 'ln_no')



## join all snapshot servicing lists

df_list =  df_201801.union(df_201802).union(df_201803).union(df_201804)\
                    .union(df_201805).union(df_201806).union(df_201807)\
                    .union(df_201808).union(df_201809).union(df_201810)\
                    .union(df_201811).union(df_201812)\
                    .union(df_201901).union(df_201902).union(df_201903)\
                    .union(df_201904).union(df_201905).union(df_201906)\
                    .union(df_201907).union(df_201908).union(df_201909)\
                    .union(df_201910).union(df_201911).union(df_201912)\
                    .union(df_202001).union(df_202002).union(df_202003)\
                    .union(df_202004).union(df_202005).union(df_202006)\
                    .union(df_202007).union(df_202008).union(df_202009)\
                    .union(df_202010).union(df_202011).union(df_202012)\
                    .union(df_202101)


In [ ]:
%%spark

## check the total count
df_list.groupBy('servicecalendardate').count().orderBy('servicecalendardate').show(5)

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2018-01-31|1561948|
|         2018-02-28|1576322|
|         2018-03-31|1593701|
|         2018-04-30|1609795|
|         2018-05-31|1624519|
+-------------------+-------+
only showing top 5 rows

In [ ]:
%%spark

df_payoff = df_list.join(df_actual, df_actual.ServicedLoanNumber == df_list.ln_no, how='left')


In [ ]:
%%spark
df_payoff.show(5)

+-------------------+----------+------------------+-----------------+--------------+-------------+
|servicecalendardate|     ln_no|ServicedLoanNumber|PaymentInFullDate|newloanpurpose|retentiontype|
+-------------------+----------+------------------+-----------------+--------------+-------------+
|         2019-02-28|3300491092|              null|             null|          null|         null|
|         2019-03-31|3300491092|              null|             null|          null|         null|
|         2020-10-31|3301516569|              null|             null|          null|         null|
|         2019-04-30|3308905158|              null|             null|          null|         null|
|         2020-12-31|3316399709|              null|             null|          null|         null|
+-------------------+----------+------------------+-----------------+--------------+-------------+
only showing top 5 rows

In [ ]:
%%spark

## create flags

def build_targets(df):

#     # Purchase Payoff
#     expr_purpayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')=='purchase'), 1.)\
#                         .otherwise(0.)
#     expr_purpayoff_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('newloanpurpose')=='purchase'), 1.)\
#                             .otherwise(0.)
#     expr_purpayoff_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('newloanpurpose')=='purchase'), 1.)\
#                             .otherwise(0.)
#     expr_purpayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')=='purchase'), 1.)\
#                             .otherwise(0.)

    # Refi Payoff
    expr_refipayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')=='refinance'), 1.)\
                        .otherwise(0.)
    expr_refipayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')=='refinance'), 1.)\
                            .otherwise(0.)

#     # Purchase/Refi Retained
#     expr_purretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24'))
#                              & (col('newloanpurpose')=='purchase') & (col('retentiontype')=='retained'), 1.)\
#                             .otherwise(0.)

#     expr_refiretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24'))
#                              & (col('newloanpurpose')!='purchase') & (col('retentiontype')=='retained'), 1.)\
#                             .otherwise(0.)

#     # Payoff
#     expr_payoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')), 1.)\
#                         .otherwise(0.)
#     expr_payoff_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')), 1.)\
#                         .otherwise(0.)
#     expr_payoff_all = when(col('PaymentInFullDate')>col('servicecalendardate'), 1.)\
#                         .otherwise(0.)

#     # Retained
#     expr_retention = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('retentiontype')=='retained'), 1.)\
#                         .otherwise(0.)
#     expr_retention_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('retentiontype')=='retained'), 1.)\
#                             .otherwise(0.)
#     expr_retention_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('retentiontype')=='retained'), 1.)\
#                             .otherwise(0.)
#     expr_retention_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) & (col('retentiontype')=='retained'), 1.)\
#                         .otherwise(0.)
#     expr_retention_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('retentiontype')=='retained'), 1.)\
#                         .otherwise(0.)

    ## the 05-31 snapshot predicts 06 propensity
    ## when calculate actual payoff, need to start from 06-01 to get the payoffs
    ## and in this logic, payoffs are counted from 07-01
    df_final = df.withColumn('startdt', F.add_months(col('servicecalendardate'), 1))\
                    .withColumn('enddt_3', F.add_months(col('servicecalendardate'), 4))\
                    .withColumn('refipayoff', expr_refipayoff)\
                    .withColumn('refipayoff_all', expr_refipayoff_all) \
                    .select('servicecalendardate', 'ln_no', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype',
                            'refipayoff', 'refipayoff_all')

    return df_final


In [ ]:
%%spark

df_final = build_targets(df_payoff)
# df_final.orderBy('servicecalendardate').show()

In [ ]:
%%spark

# df_payoff.groupBy('servicecalendardate').count().sort('servicecalendardate').show()
df_final.groupBy('servicecalendardate', 'refipayoff').count().sort('servicecalendardate', 'refipayoff').show(200)

+-------------------+----------+-------+
|servicecalendardate|refipayoff|  count|
+-------------------+----------+-------+
|         2018-01-31|       0.0|1534331|
|         2018-01-31|       1.0|  27617|
|         2018-02-28|       0.0|1549591|
|         2018-02-28|       1.0|  26731|
|         2018-03-31|       0.0|1566304|
|         2018-03-31|       1.0|  27397|
|         2018-04-30|       0.0|1580713|
|         2018-04-30|       1.0|  29082|
|         2018-05-31|       0.0|1595081|
|         2018-05-31|       1.0|  29438|
|         2018-06-30|       0.0|1607495|
|         2018-06-30|       1.0|  30246|
|         2018-07-31|       0.0|1624353|
|         2018-07-31|       1.0|  28068|
|         2018-08-31|       0.0|1641296|
|         2018-08-31|       1.0|  27906|
|         2018-09-30|       0.0|1657037|
|         2018-09-30|       1.0|  26546|
|         2018-10-31|       0.0|1671440|
|         2018-10-31|       1.0|  28070|
|         2018-11-30|       0.0|1682143|
|         2018-1

In [ ]:
%%spark

# df_payoff.groupBy('servicecalendardate').count().sort('servicecalendardate').show()
df_final.groupBy('servicecalendardate', 'newloanpurpose').count().sort('servicecalendardate', 'newloanpurpose').show(100)

+-------------------+--------------+-------+
|servicecalendardate|newloanpurpose|  count|
+-------------------+--------------+-------+
|         2018-01-31|          null| 806679|
|         2018-01-31|      purchase| 171615|
|         2018-01-31|     refinance| 568293|
|         2018-01-31|       unknown|  15361|
|         2018-02-28|          null| 818182|
|         2018-02-28|      purchase| 171254|
|         2018-02-28|     refinance| 571651|
|         2018-02-28|       unknown|  15235|
|         2018-03-31|          null| 831826|
|         2018-03-31|      purchase| 169355|
|         2018-03-31|     refinance| 577530|
|         2018-03-31|       unknown|  14990|
|         2018-04-30|          null| 844267|
|         2018-04-30|      purchase| 167001|
|         2018-04-30|     refinance| 583721|
|         2018-04-30|       unknown|  14806|
|         2018-05-31|          null| 856531|
|         2018-05-31|      purchase| 163581|
|         2018-05-31|     refinance| 589908|
|         

In [ ]:
%%spark

df_final.write.parquet('/uat/DataScience/Maestro/Mover Refi/Actual_payoff_20210405.parquet')

# Performance Check

In [ ]:
%%spark

df_final = spark.read.parquet(actual_path + 'Actual_payoff_20200709.parquet')

In [ ]:
%%spark

df_final.groupBy('servicecalendardate').count().sort(desc('servicecalendardate')).show()

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-06-30|1931387|
|         2020-05-31|1889579|
|         2020-04-30|1899099|
|         2020-03-31|1867297|
|         2020-02-29|1846864|
|         2020-01-31|1824706|
|         2019-12-31|1802243|
|         2019-11-30|1807991|
|         2019-10-31|1787277|
|         2019-09-30|1763391|
|         2019-08-31|1856202|
|         2019-07-31|1839005|
|         2019-06-30|1822001|
|         2019-05-31|1806604|
|         2019-04-30|1788444|
|         2019-03-31|1770015|
|         2019-02-28|1752593|
|         2019-01-31|1738794|
|         2018-12-31|1726017|
|         2018-11-30|1713615|
+-------------------+-------+
only showing top 20 rows

In [ ]:
%%spark
result_path_refi = '/dev/projects/retention_models/refi_payoff/training/results/'
df_score = spark.read.csv(result_path_refi + 'pred_refiV1_4_feb20_20200514.csv', header=True)\
                            .withColumn('servicecalendardate', F.lit('2020-02-29'))\
                            .withColumn('score', col('gbtProb').cast('double'))
df_actual = spark.read.parquet(actual_path + 'Actual_payoff_20200610.parquet')
df_all = df_score.join(df_actual, on=['servicecalendardate','ln_no'], how = 'left')

In [ ]:
%%spark

df_all.show(2)

+-------------------+----------+------------------+-------------------+-------------------+-------------------+-----------------+--------------+-------------+--------------+------------------+-------------------+------------------+----------+--------------+------------+-------------+------+---------+----------+--------+------------+-------------+-----------+------------+
|servicecalendardate|     ln_no|        logRegProb|        randForProb|            gbtProb|              score|PaymentInFullDate|newloanpurpose|retentiontype|purchasepayoff|purchasepayoff_3_6|purchasepayoff_6_12|purchasepayoff_all|refipayoff|refipayoff_all|purretain_24|refiretain_24|payoff|payoff_24|payoff_all|retained|retained_3_6|retained_6_12|retained_24|retained_all|
+-------------------+----------+------------------+-------------------+-------------------+-------------------+-----------------+--------------+-------------+--------------+------------------+-------------------+------------------+----------+----------

In [ ]:
%%spark

from pyspark.sql import Window
from pyspark.ml.feature import QuantileDiscretizer

df1 = QuantileDiscretizer(numBuckets=10, inputCol='score', outputCol='ntile', relativeError=0.0001, handleInvalid='error')\
                    .fit(df_all).transform(df_all)
df1 = df1.withColumn('ntile', (10 - col('ntile')).cast('int'))

windown_cumsum = Window.orderBy('ntile').rangeBetween(Window.unboundedPreceding, 0)
total_target = df1.select(F.sum('refipayoff')).collect()[0][0]

df_out = df1.groupBy('ntile')\
                    .agg(F.sum('refipayoff').alias('refipayoff'),
                         F.count('ln_no').alias('count'))\
                    .sort('ntile')

In [ ]:
%%spark

df_out.show()

+-----+----------+------+
|ntile|refipayoff| count|
+-----+----------+------+
|    1|   24391.0|184244|
|    2|   17495.0|184826|
|    3|   13755.0|184512|
|    4|   12406.0|184950|
|    5|   10705.0|184638|
|    6|    8762.0|184511|
|    7|    7050.0|184882|
|    8|    5453.0|184432|
|    9|    3507.0|184936|
|   10|     997.0|184933|
+-----+----------+------+